<a href="https://colab.research.google.com/github/as9786/ComputerVision/blob/main/%EA%B8%B0%EB%B3%B8%ED%95%A9%EC%84%B1%EA%B3%B1%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 합성곱 신경망 구현

In [1]:
import tensorflow as tf
import numpy as np

# 초매개변수

In [2]:
EPOCHS = 10

# 모형 정의

In [3]:
class ConvNet(tf.keras.Model):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.sequence = list()
    self.sequence.append(tf.keras.layers.Conv2D(16,(3,3),padding='same',activation='relu')) # 28 x 28 x 16
    self.sequence.append(tf.keras.layers.Conv2D(16,(3,3),padding='same',activation='relu'))
    self.sequence.append(tf.keras.layers.MaxPool2D((2,2))) # 14 x 14 x 16
    self.sequence.append(tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu')) # 14 x 14 x 32
    self.sequence.append(tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'))
    self.sequence.append(tf.keras.layers.MaxPool2D((2,2))) # 7 x 7 x 32
    self.sequence.append(tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu')) # 7 x 7 x 64
    self.sequence.append(tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu'))
    self.sequence.append(tf.keras.layers.Flatten())
    self.sequence.append(tf.keras.layers.Dense(2048,activation='relu'))
    self.sequence.append(tf.keras.layers.Dense(10,activation='softmax'))

  def call(self,x,training=False,mask=None): # training이라는 인자는 후에 dropout과 같은 부분의 인자를 받기 위해 필요요
    for layer in self.sequence:
      x = layer(x)
    return x

#Train and Test

## Dataset

In [4]:
mnist = tf.keras.datasets.mnist

(X_train,y_train),(X_test,y_test) = mnist.load_data()

X_train, X_test = X_train / 255., X_test / 255.

X_train = X_train[...,tf.newaxis].astype(np.float32)
X_test = X_test[...,tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(32)

11490434/11490434 [==============================] - 0s 0us/step


## 학습 환경 정의

### 모형 생성, 손실함수, 최적화, 평가지표

In [5]:
model = ConvNet() 

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [6]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images) 
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)

@tf.function
def test_step(model,images,labels,loss_object,test_loss,test_accuracy): 
  predictions = model(images)
  
  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)


### 학습

In [7]:
for epoch in range(EPOCHS): 
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds: 
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch {}, Loss : {}, Accuracy : {}, Test Loss : {}, Test Accuracy : {}'
  print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(),
                        test_accuracy.result() * 100))

  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


Epoch 1, Loss : 0.11389899253845215, Accuracy : 96.55667114257812, Test Loss : 0.061018362641334534, Test Accuracy : 98.2699966430664
Epoch 2, Loss : 0.04620257392525673, Accuracy : 98.62667083740234, Test Loss : 0.03478148207068443, Test Accuracy : 98.98999786376953
Epoch 3, Loss : 0.03385720029473305, Accuracy : 98.95832824707031, Test Loss : 0.03542362153530121, Test Accuracy : 98.8499984741211
Epoch 4, Loss : 0.0276486836373806, Accuracy : 99.19000244140625, Test Loss : 0.030106758698821068, Test Accuracy : 99.08999633789062
Epoch 5, Loss : 0.024283550679683685, Accuracy : 99.22833251953125, Test Loss : 0.033690840005874634, Test Accuracy : 99.11000061035156
Epoch 6, Loss : 0.01996949501335621, Accuracy : 99.37999725341797, Test Loss : 0.026677360758185387, Test Accuracy : 99.36000061035156
Epoch 7, Loss : 0.018087811768054962, Accuracy : 99.46333312988281, Test Loss : 0.03065625950694084, Test Accuracy : 99.13999938964844
Epoch 8, Loss : 0.015954812988638878, Accuracy : 99.5233383